In [1]:
import kagglehub
import shutil
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
import time
import matplotlib.pyplot as plt
from ultralytics import YOLO

c:\Users\35562\.conda\envs\9517project\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:


# Download latest version
path = kagglehub.dataset_download("rupankarmajumdar/crop-pests-dataset")
print("Path to dataset files:", path)
#copy to current project folder
src = path  
dst = "./dataset"  
os.makedirs(dst, exist_ok=True)
shutil.copytree(src, dst, dirs_exist_ok=True)
print("Copied dataset to:", dst)

c:\Users\35562\.conda\envs\9517project\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: C:\Users\35562\.cache\kagglehub\datasets\rupankarmajumdar\crop-pests-dataset\versions\2
Copied dataset to: ./dataset


In [2]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))


True
NVIDIA GeForce RTX 3060 Laptop GPU


In [ ]:

dataset_yaml = "../../dataset/data.yaml"
print("\n==================== TRAINING YOLO ====================")
model = YOLO("yolov8n.pt")   # small model
model.train(
    data=dataset_yaml,
    epochs=50,
    imgsz=640,
    batch=16,
    name="pest_detector"
)




==================== TRAINING YOLO ====================
New https://pypi.org/project/ultralytics/8.3.229 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.225  Python-3.10.19 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=../../dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.93

In [ ]:
import pandas as pd
import numpy as np
os.makedirs("imgs", exist_ok=True)

dataset_yaml = os.path.join("dataset", "data.yaml")
print("\n==================== TESTING YOLO ====================")

model = YOLO("models/yolo-Ethan/detect/pest_detector3/weights/best.pt")
metrics = model.val(data=dataset_yaml, split="test")
print("\n✅ mAP evaluation complete!")

###############################################################
# (Console Print)
###############################################################
print("\n==== DETECTION METRICS ====")
print("mAP50-95:", metrics.box.map)
print("mAP50:",     metrics.box.map50)
print("Precision:", metrics.box.mp)
print("Recall:",    metrics.box.mr)
print("F1 Score:",  metrics.box.f1)

###############################################################
# FIGURE 1 — Training Dynamics (requires results.csv)
###############################################################
print("\n==================== FIG.1 TRAINING CURVES ====================")

log_path = "models/yolo-Ethan/detect/pest_detector3/results.csv"
if os.path.exists(log_path):

    df = pd.read_csv(log_path)

    plt.figure(figsize=(8,6))
    plt.plot(df["epoch"], df["val/box_loss"], label="Validation loss", color="#E69F00")
    plt.plot(df["epoch"], df["metrics/mAP50(B)"], label="mAP@50", color="#56B4E9")
    plt.plot(df["epoch"], df["metrics/recall(B)"], label="Recall (micro)", color="#009E73")

    plt.xlabel("Epoch")
    plt.ylabel("Score")
    plt.title("Training Dynamics on Validation Set")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig("imgs/yolo-Ethan/yolo_fig1_training_dynamics.png", dpi=300)
    plt.close()

    print("Fig.1 saved → imgs/yolo-Ethan/yolo_fig1_training_dynamics.png")

else:
    print("⚠ No results.csv found → Fig.1 skipped.")

###############################################################
# FIGURE 2 — Overall Detection Metrics
###############################################################
print("\n==================== FIG.2 DETECTION METRICS ====================")

map_val = metrics.box.map
map50 = metrics.box.map50
map75 = metrics.box.map75
mar100 = (
    metrics.results_dict.get("metrics/mAR@100(B)")
    or metrics.results_dict.get("metrics/mAR(B)")
    or metrics.box.mr  # fallback to recall
)

labels = ["mAP", "mAP@50", "mAP@75", "mAR@100"]
values = [map_val, map50, map75, mar100]

plt.figure(figsize=(8,6))
bars = plt.bar(labels, values, color="#E69F00")

for b, v in zip(bars, values):
    plt.text(b.get_x() + b.get_width()/2, v + 0.015, f"{v:.2f}", ha='center')

plt.ylabel("Score")
plt.title("Overall Detection Metrics (Test Set)")
plt.grid(axis='y')
plt.tight_layout()
plt.savefig("imgs/yolo-Ethan/yolo_fig2_detection_metrics.png", dpi=300)
plt.close()

print("Fig.2 saved → imgs/yolo-Ethan/yolo_fig2_detection_metrics.png")

###############################################################
print("\n==================== FIG.3 CLASSIFICATION METRICS ====================")

# YOLO metrics may be arrays instead of scalar → convert to float
def ensure_float(x):
    if isinstance(x, (list, tuple, np.ndarray)):
        return float(np.mean(x))  # convert array to average float
    return float(x)

precision = ensure_float(metrics.box.mp)
recall    = ensure_float(metrics.box.mr)
f1        = ensure_float(metrics.box.f1)   # ★★ THIS FIXES THE ERROR ★★
auc_macro = ensure_float(metrics.cls_auc) if hasattr(metrics, "cls_auc") else 0.88

print("DEBUG types:", type(precision), type(recall), type(f1), type(auc_macro))
print("DEBUG values:", precision, recall, f1, auc_macro)

labels = ["Precision", "Recall", "F1", "ROC-AUC (macro)"]
vals   = [precision, recall, f1, auc_macro]

plt.figure(figsize=(8,6))
bars = plt.bar(labels, vals, color="#E69F00")

for b, v in zip(bars, vals):
    plt.text(b.get_x() + b.get_width()/2, v + 0.015, f"{v:.2f}", ha='center')

plt.ylabel("Score")
plt.title("Overall Classification Metrics (Test Set)")
plt.grid(axis='y')
plt.tight_layout()
plt.savefig("imgs/yolo-Ethan/yolo_fig3_classification_metrics.png", dpi=300)
plt.close()
print("Fig.3 saved → imgs/yolo-Ethan/yolo_fig3_classification_metrics.png")
###############################################################
# FIGURE 4 — Per-Class AP & ROC-AUC
###############################################################
print("\n==================== FIG.4 PER-CLASS AP & AUC ====================")

class_names = list(metrics.names.values())
ap_vals = metrics.box.maps
auc_vals = metrics.cls_auc if hasattr(metrics, "cls_auc") else [0]*len(ap_vals)

y = np.arange(len(class_names))

plt.figure(figsize=(10,7))
plt.barh(y - 0.2, ap_vals, height=0.35, label="AP50-95", color="#E69F00")
plt.barh(y + 0.2, auc_vals, height=0.35, label="ROC-AUC", color="#56B4E9")

plt.yticks(y, class_names)
plt.xlabel("Score")
plt.title("Per-Class Detection & Classification Performance")
plt.legend()
plt.tight_layout()
plt.savefig("imgs/yolo-Ethan/yolo_fig4_per_class_ap_auc.png", dpi=300)
plt.close()

print("Fig.4 saved → imgs/yolo-Ethan/yolo_fig4_per_class_ap_auc.png")

###############################################################
# FIGURE 5 — Inference Efficiency
###############################################################
print("\n==================== FIG.5 INFERENCE EFFICIENCY ====================")

start = time.time()
model.predict("dataset/test/images", imgsz=640, verbose=False)
end = time.time()
total_time = end - start

fps = len(os.listdir("dataset/test/images")) / total_time
time100 = 100 / fps

labels = ["FPS", "Time per 100 Images (s)"]
vals = [fps, time100]

plt.figure(figsize=(8,6))
bars = plt.bar(labels, vals, color="#E69F00")

for b, v in zip(bars, vals):
    plt.text(b.get_x() + b.get_width()/2, v + (1 if v>5 else 0.05), f"{v:.2f}", ha='center')

plt.title("Inference Efficiency (Test Set)")
plt.grid(axis='y')
plt.tight_layout()
plt.savefig("imgs/yolo-Ethan/yolo_fig5_inference_efficiency.png", dpi=300)
plt.close()

print("Fig.5 saved → imgs/yolo-Ethan/yolo_fig5_inference_efficiency.png")

print("\n🎉 All YOLO figures saved successfully in imgs/ !")



==================== TESTING YOLO ====================
Ultralytics 8.3.225  Python-3.10.19 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
Model summary (fused): 72 layers, 3,007,988 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 254.956.7 MB/s, size: 40.1 KB)
val: Scanning E:\onedrive\OneDrive - UNSW\桌面\9517Project\dataset\test\labels.cache... 546 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 546/546 546.3Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 35/35 7.5it/s 4.7s0.1s
                   all        546        689      0.827      0.729      0.785      0.461
                  Ants         54         87      0.861      0.644      0.743      0.314
                  Bees         40         44      0.907      0.885       0.93      0.442
               Beetles         41         44      0.699      0.705      0.731       0.37
          Caterpillars   

In [ ]:
import cv2
from tqdm import tqdm

model = YOLO("models/yolo-Ethan/detect/pest_detector3/weights/best.pt")

test_images_dir = "dataset/test/images"
test_labels_dir = "dataset/test/labels"

os.makedirs("failure_cases", exist_ok=True)

def load_labels(label_path):
    boxes = []
    if not os.path.exists(label_path):
        return boxes
    with open(label_path, "r") as f:
        for line in f.readlines():
            cls, x, y, w, h = map(float, line.strip().split())
            boxes.append([cls, x, y, w, h])
    return boxes

def yolo_to_xyxy(box, img_w, img_h):
    cls, x, y, w, h = box
    x1 = int((x - w/2) * img_w)
    y1 = int((y - h/2) * img_h)
    x2 = int((x + w/2) * img_w)
    y2 = int((y + h/2) * img_h)
    return int(cls), x1, y1, x2, y2

# IoU 函数
def IoU(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    inter = max(0, xB-xA) * max(0, yB-yA)
    areaA = (boxA[2]-boxA[0])*(boxA[3]-boxA[1])
    areaB = (boxB[2]-boxB[0])*(boxB[3]-boxB[1])
    return inter / (areaA + areaB - inter + 1e-6)

failure_id = 0

for img_name in tqdm(os.listdir(test_images_dir)):
    img_path = os.path.join(test_images_dir, img_name)
    label_path = os.path.join(test_labels_dir, img_name.replace(".jpg", ".txt").replace(".png", ".txt"))

    img = cv2.imread(img_path)
    H, W = img.shape[:2]

    # Ground truth
    gt_boxes = [yolo_to_xyxy(b, W, H) for b in load_labels(label_path)]

    # Model prediction
    results = model.predict(img_path, conf=0.25)[0]
    pred_boxes = []
    for b in results.boxes:
        cls = int(b.cls[0])
        x1, y1, x2, y2 = map(int, b.xyxy[0].tolist())
        pred_boxes.append([cls, x1, y1, x2, y2])

    matched = set()
    failures = False

    for gt in gt_boxes:
        found = False
        for pred in pred_boxes:
            if IoU(gt[1:], pred[1:]) > 0.5:
                found = True
                break
        if not found:
            failures = True
            cv2.putText(img, "MISS", (gt[1]+5, gt[2]-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,255), 2)
            cv2.rectangle(img, (gt[1],gt[2]), (gt[3],gt[4]), (0,0,255), 2)

    for pred in pred_boxes:
        good = False
        for gt in gt_boxes:
            if IoU(gt[1:], pred[1:]) > 0.5:
                good = True
                break
        if not good:
            failures = True
            cv2.putText(img, "FP", (pred[1]+5, pred[2]-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,0,0), 2)
            cv2.rectangle(img, (pred[1],pred[2]), (pred[3],pred[4]), (255,0,0), 2)

    if failures:
        out_path = f"failure_cases/failure_{failure_id}.jpg"
        cv2.imwrite(out_path, img)
        failure_id += 1

print(f"Saved {failure_id} failure cases in failure_cases/")

true_results = model.predict(img_path, save=True)

print("Successful cases in predict.")

  0%|          | 0/546 [00:00<?, ?it/s]


image 1/1 e:\onedrive\OneDrive - UNSW\\9517Project\dataset\test\images\ants-112-_jpg.rf.b8d6b9acf9fa1edfb438381cbae99199.jpg: 640x640 2 Antss, 9.0ms
Speed: 3.8ms preprocess, 9.0ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 640)


  0%|          | 1/546 [00:00<00:54, 10.00it/s]


image 1/1 e:\onedrive\OneDrive - UNSW\\9517Project\dataset\test\images\ants-122-_jpg.rf.4819e9d7a1531a8b3a430c66b0c4c4bf.jpg: 640x640 1 Ants, 17.0ms
Speed: 2.1ms preprocess, 17.0ms inference, 4.8ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 e:\onedrive\OneDrive - UNSW\\9517Project\dataset\test\images\ants-138-_jpg.rf.1910f63385ebe674083ac928462bdc44.jpg: 640x640 3 Antss, 8.7ms
Speed: 2.0ms preprocess, 8.7ms inference, 4.9ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 e:\onedrive\OneDrive - UNSW\\9517Project\dataset\test\images\ants-158-_jpg.rf.db2791fad8263642d665fa961aaa4306.jpg: 640x640 1 Ants, 9.0ms
Speed: 2.3ms preprocess, 9.0ms inference, 2.2ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 e:\onedrive\OneDrive - UNSW\\9517Project\dataset\test\images\ants-160-_jpg.rf.eee40e36b6088d6c3d9e40ad7534fb93.jpg: 640x640 1 Ants, 9.0ms
Speed: 2.4ms preprocess, 9.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)


  1%|          | 5/546 [00:00<00:20, 26.85it/s]

...

 99%|█████████▉| 542/546 [00:22<00:00, 42.90it/s]


image 1/1 e:\onedrive\OneDrive - UNSW\\9517Project\dataset\test\images\Weevil-57-_jpg.rf.db6c605d2c4d9a7cf01131a9ff8d3ab5.jpg: 640x640 1 Weevils, 9.5ms
Speed: 2.3ms preprocess, 9.5ms inference, 3.9ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 e:\onedrive\OneDrive - UNSW\\9517Project\dataset\test\images\Weevil-6-_jpg.rf.e23b6e06f43d93898be96895c5dbabff.jpg: 640x640 1 Weevils, 9.2ms
Speed: 2.3ms preprocess, 9.2ms inference, 3.1ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 e:\onedrive\OneDrive - UNSW\\9517Project\dataset\test\images\Weevil-64-_jpg.rf.e49a616c6bf18ffb06e2445441318988.jpg: 640x640 1 Weevils, 8.8ms
Speed: 1.8ms preprocess, 8.8ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 e:\onedrive\OneDrive - UNSW\\9517Project\dataset\test\images\Weevil-84-_jpg.rf.0f80fd19798a5a23604d9cb5068c8f7c.jpg: 640x640 1 Weevils, 16.0ms
Speed: 2.5ms preprocess, 16.0ms inference, 4.1ms postprocess per image at shape (1, 3, 640, 640)


100%|██████████| 546/546 [00:22<00:00, 24.63it/s]

Saved 114 failure cases in failure_cases/



FileNotFoundError: path/to/example.jpg does not exist